In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Assignments/HRDataset_v14.csv")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 36 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Employee_Name               311 non-null    object 
 1   EmpID                       311 non-null    int64  
 2   MarriedID                   311 non-null    int64  
 3   MaritalStatusID             311 non-null    int64  
 4   GenderID                    311 non-null    int64  
 5   EmpStatusID                 311 non-null    int64  
 6   DeptID                      311 non-null    int64  
 7   PerfScoreID                 311 non-null    int64  
 8   FromDiversityJobFairID      311 non-null    int64  
 9   Salary                      311 non-null    int64  
 10  Termd                       311 non-null    int64  
 11  PositionID                  311 non-null    int64  
 12  Position                    311 non-null    object 
 13  State                       311 non

In [4]:
data = data.drop(['ManagerID','DateofTermination', 'Employee_Name','EmpID','DOB','Sex','MaritalDesc','CitizenDesc','TermReason','ManagerName','RecruitmentSource','LastPerformanceReview_Date'], axis = 1)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   MarriedID               311 non-null    int64  
 1   MaritalStatusID         311 non-null    int64  
 2   GenderID                311 non-null    int64  
 3   EmpStatusID             311 non-null    int64  
 4   DeptID                  311 non-null    int64  
 5   PerfScoreID             311 non-null    int64  
 6   FromDiversityJobFairID  311 non-null    int64  
 7   Salary                  311 non-null    int64  
 8   Termd                   311 non-null    int64  
 9   PositionID              311 non-null    int64  
 10  Position                311 non-null    object 
 11  State                   311 non-null    object 
 12  Zip                     311 non-null    int64  
 13  HispanicLatino          311 non-null    object 
 14  RaceDesc                311 non-null    ob

In [6]:
data = pd.get_dummies(data,['Position','State','HispanicLatino','RaceDesc','DateofHire','EmploymentStatus','Department','PerformanceScore'])

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Columns: 200 entries, MarriedID to PerformanceScore_PIP
dtypes: float64(1), int64(15), uint8(184)
memory usage: 94.9 KB


In [8]:
X = data.drop(['Salary'], axis = 1)
y = data[['Salary']]

### Randomized Search

Using Linear Regression, SGD Regressor, Ridge Regression, Support Vector Regression, and Decision Tree Regression

In [9]:
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

####Splitting

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

####Part 1: Listing Parameters

In [11]:
model_params ={
      'SVR': {
          'model': SVR(gamma = 'auto'),
          'params': {
              'C': [0.1,1,10,20,100],
              'kernel': ['rbf', 'linear','poly']

          }
      },
      'SGD': {
          'model': SGDRegressor(max_iter = 1000, tol = 1e-3),
          'params': { 
              'eta0':[0.001,0.01,0.1,0.5,1,10,20,100]

          }
      },
      'LinearRegression':{
          'model': LinearRegression(),
          'params':{}
      },
      'Ridge_Regressor':{
          'model': Ridge(),
          'params': {'alpha' : [0.05,0.01,0.1,0.5,1,10,100,200],
                     'max_iter': [100,500,1000],
                     }
      },
      'DecisionTreeRegressor':{
          'model': DecisionTreeRegressor(),          
          'params':{'max_depth':[3,5],
                    },
      }

}

####Scaling Data

In [12]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()

In [13]:
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [14]:
y_train = sc_y.fit_transform(y_train)
y_test = sc_y.transform(y_test)

####Part 2: Random Search

In [15]:
from sklearn.model_selection import RandomizedSearchCV
scores = []
for model_name, model_parameter in model_params.items():
  clf = RandomizedSearchCV(model_parameter['model'],model_parameter['params'],
                           cv =6,
                           return_train_score = False,
                           n_iter = 5)
                
  clf.fit(X_train,y_train)

  scores.append({
    'model': model_name,
    'best_scores': clf.best_score_,
    'best_params':clf.best_params_
})

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

In [16]:
scores

[{'model': 'SVR',
  'best_scores': 0.36109428282369976,
  'best_params': {'kernel': 'linear', 'C': 100}},
 {'model': 'SGD',
  'best_scores': -1512116954170052.0,
  'best_params': {'eta0': 0.1}},
 {'model': 'LinearRegression',
  'best_scores': -6.751802415580256e+27,
  'best_params': {}},
 {'model': 'Ridge_Regressor',
  'best_scores': 0.29679577219523356,
  'best_params': {'max_iter': 1000, 'alpha': 0.1}},
 {'model': 'DecisionTreeRegressor',
  'best_scores': 0.3773948420588949,
  'best_params': {'max_depth': 5}}]

In [25]:
df = pd.DataFrame(scores,columns = ['model','best_scores','best_params'])
df

,model,best_scores,best_params
0,SVR,3.610943e-01,"{'kernel': 'linear', 'C': 100}"
1,SGD,-1.512117e+15,{'eta0': 0.1}
2,LinearRegression,-6.751802e+27,{}
3,Ridge_Regressor,2.967958e-01,"{'max_iter': 1000, 'alpha': 0.1}"
4,DecisionTreeRegressor,3.773948e-01,{'max_depth': 5}
